# Import

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error as mae
import matplotlib.pyplot as plt
import seaborn as sns
from preprocess import preprocess, process, league_train_test_split, league_fusion, league_split, league_fusion_cross_val, preprocess_fusion, process_fusion
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression

D:\ProgramFiles\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\ProgramFiles\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
D:\ProgramFiles\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Data

In [2]:
%%time
data = pd.read_csv('Kaioken_x200.csv')

CPU times: total: 26.4 s
Wall time: 27.1 s


In [3]:
data.drop(['club_id_'+str(i) for i in range(1, 15)], axis=1, inplace=True)

In [4]:
data

,league_id,dynamic_payment_segment_1,cohort_season_1,avg_age_top_11_players_1,avg_stars_top_11_players_1,avg_stars_top_14_players_1,avg_training_factor_top_11_players_1,days_active_last_28_days_1,league_match_watched_count_last_28_days_1,session_count_last_28_days_1,...,league_rank_5,league_rank_6,league_rank_7,league_rank_8,league_rank_9,league_rank_10,league_rank_11,league_rank_12,league_rank_13,league_rank_14
0,2906404,0) NonPayer,1,24,3.796727,3.569619,0.531916,2,0,8,...,1,2,11,3,13,7,8,6,12,5
1,2909517,2) Minnow,8,24,5.785891,5.540248,0.472145,28,15,188,...,7,13,11,12,10,8,3,6,5,2
2,2906572,0) NonPayer,3,23,2.805891,2.697943,0.648875,0,0,0,...,8,6,13,11,9,1,14,5,12,2
3,2912779,3) Dolphin,93,25,6.985443,6.692543,0.699977,28,18,172,...,10,11,14,9,7,4,6,8,2,12
4,2912586,1) ExPayer,141,29,5.546703,5.379000,0.556755,28,0,41,...,3,13,12,1,11,10,5,4,9,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790195,2912266,1) ExPayer,83,23,4.113042,3.891362,0.560736,13,2,22,...,1,8,2,7,3,4,5,12,13,14
790196,2908479,0) NonPayer,2,24,4.252715,4.130314,0.476840,0,0,0,...,6,14,5,11,13,4,1,12,9,3
790197,2911934,0) NonPayer,63,21,4.532533,4.388295,0.694876,10,0,14,...,13,10,12,6,11,2,1,3,8,4
790198,2907606,0) NonPayer,1,25,3.642376,3.504410,0.538899,3,0,7,...,12,11,1,5,9,4,8,6,14,10


# Data split

In [5]:
targets = data.columns[-14:]
X = data.drop(targets, axis=1)
Y = data[targets]

In [6]:
del data

In [7]:
%%time
X = preprocess_fusion(X).drop('league_id', axis=1)

CPU times: total: 43.3 s
Wall time: 43.6 s


In [12]:
X

,cohort_season_1,avg_age_top_11_players_1,avg_stars_top_11_players_1,avg_stars_top_14_players_1,avg_training_factor_top_11_players_1,days_active_last_28_days_1,league_match_watched_count_last_28_days_1,session_count_last_28_days_1,playtime_last_28_days_1,league_match_won_count_last_28_days_1,...,Android Tablet_14,Flash FB Canvas_14,Flash TE Site_14,UniversalWindows PC_14,WebGL FB Canvas_14,WebGL TE Site_14,WebGL or Flash FB Canvas_14,WebGL or Flash TE Site_14,iOS Phone_14,iOS Tablet_14
0,1,24,3.796727,3.569619,0.531916,2,0,8,5522082,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8,24,5.785891,5.540248,0.472145,28,15,188,130030004,14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,23,2.805891,2.697943,0.648875,0,0,0,0,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,93,25,6.985443,6.692543,0.699977,28,18,172,49452393,24,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,141,29,5.546703,5.379000,0.556755,28,0,41,19420865,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790195,83,23,4.113042,3.891362,0.560736,13,2,22,6000565,18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
790196,2,24,4.252715,4.130314,0.476840,0,0,0,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
790197,63,21,4.532533,4.388295,0.694876,10,0,14,7164442,10,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
790198,1,25,3.642376,3.504410,0.538899,3,0,7,607546,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Model training

### XGBoost

In [9]:
multi_xgb_regression = MultiOutputRegressor(XGBRegressor(max_depth=2, n_estimators=1024, device='cuda'), n_jobs=2)

In [13]:
%%time
multi_xgb_regression.fit(X, Y)
for i, _ in enumerate(multi_xgb_regression.estimators_):
    multi_xgb_regression.estimators_[i].set_params(device='cpu')

CPU times: total: 11.1 s
Wall time: 22min 34s


# Test data prediction

In [27]:
test_data = pd.read_csv('jobfair_test.csv')

In [28]:
test_data.drop(['season', 'registration_country'], axis=1, inplace=True)

In [29]:
test_data

,club_id,league_id,dynamic_payment_segment,cohort_season,avg_age_top_11_players,avg_stars_top_11_players,avg_stars_top_14_players,avg_training_factor_top_11_players,days_active_last_28_days,league_match_watched_count_last_28_days,session_count_last_28_days,playtime_last_28_days,registration_platform_specific,league_match_won_count_last_28_days,training_count_last_28_days,global_competition_level,tokens_spent_last_28_days,tokens_stash,rests_stash,morale_boosters_stash
0,14542747,2951383,0) NonPayer,1,23,4.295345,4.100333,0.469406,8,0,10,2580346,iOS Phone,10,6,NaN,82,18,177,106
1,11019672,2954266,0) NonPayer,21,24,4.704727,4.484933,0.317702,28,5,61,15521681,Android Phone,16,26,7.0,153,65,1030,717
2,14358567,2951259,0) NonPayer,2,22,2.923867,2.819171,0.669540,0,0,0,0,Android Phone,7,0,NaN,0,138,156,98
3,14644461,2949546,0) NonPayer,1,22,3.114776,2.977457,0.639923,3,0,3,1136208,Android Phone,2,1,NaN,2,43,43,25
4,13718978,2952772,0) NonPayer,6,23,4.194497,4.114257,0.486229,26,3,80,43655679,Android Phone,22,46,1.0,100,30,45,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60265,14614619,2950335,3) Dolphin,1,22,5.717321,5.430162,0.455917,28,22,535,264650737,iOS Phone,25,183,5.0,346,50,25,170
60266,13805591,2955052,0) NonPayer,5,21,4.079200,3.851924,0.645815,1,0,3,543672,Android Phone,12,1,2.0,0,12,188,148
60267,14559860,2953561,0) NonPayer,1,23,4.839515,4.643467,0.459347,20,3,51,19993567,Android Tablet,16,40,2.0,145,3,9,173
60268,14671631,2950158,0) NonPayer,1,26,4.882400,4.596143,0.402482,8,4,76,49796851,WebGL TE Site,6,69,2.0,109,2,57,84


In [30]:
X_test = league_fusion(test_data)

In [31]:
X_test = process_fusion(X_test)

In [32]:
X_test

,league_id,club_id_1,cohort_season_1,avg_age_top_11_players_1,avg_stars_top_11_players_1,avg_stars_top_14_players_1,avg_training_factor_top_11_players_1,days_active_last_28_days_1,league_match_watched_count_last_28_days_1,session_count_last_28_days_1,...,Android Tablet_14,Flash FB Canvas_14,Flash TE Site_14,UniversalWindows PC_14,WebGL FB Canvas_14,WebGL TE Site_14,WebGL or Flash FB Canvas_14,WebGL or Flash TE Site_14,iOS Phone_14,iOS Tablet_14
0,2955761,14159129,3,21,4.877758,4.748781,0.528695,22,1,59,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2950855,14448632,2,22,4.179903,3.927029,0.660617,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2954974,9566557,30,21,4.241394,4.163400,0.839051,9,1,44,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2955055,13778312,5,25,4.123624,3.922771,0.232181,2,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2951043,14507256,2,22,3.655903,3.362610,0.573922,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4300,2956007,14221726,3,26,5.150412,4.976867,0.399433,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4301,2951712,14243488,3,27,3.724424,3.548543,0.616774,5,0,13,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4302,2953066,14218183,3,23,5.274945,5.175286,0.489752,24,1,103,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4303,2955744,6027793,90,29,6.313176,6.181219,0.490270,23,2,40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [33]:
club_ids = pd.DataFrame()
for i in range(1, 15):
    club_ids[f'club_id_{i}'] = X_test[f'club_id_{i}']
    X_test.drop(f'club_id_{i}', axis=1, inplace=True)

In [34]:
club_ids

,club_id_1,club_id_2,club_id_3,club_id_4,club_id_5,club_id_6,club_id_7,club_id_8,club_id_9,club_id_10,club_id_11,club_id_12,club_id_13,club_id_14
0,14159129,14016945,7733794,11627008,12733754,14217178,5034852,12032840,14402089,14607166,10436626,13786106,14380815,7595972
1,14448632,14513877,14429993,14289175,14509318,14456562,14508927,14322987,14424719,14262738,14455900,14483626,14512788,14262129
2,9566557,14351617,9503891,14585763,13824331,14531835,14222316,14371592,14027510,13218601,14210439,9136493,14153678,11645846
3,13778312,14602591,13836404,14518080,14337090,14355079,13088732,14528353,13970074,14583924,8240357,14548636,9107278,13816928
4,14507256,14481670,14483743,14514489,14452652,14466123,14471603,14476089,14507061,14424063,14230495,14283161,14434536,14290982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4300,14221726,6481187,6031755,14221405,8072780,12387225,7846700,6023744,13648995,7144040,5966913,8544303,14365570,11418161
4301,14243488,14250521,14261918,14254458,14521215,14279126,14058030,14255735,14254779,14229298,14249607,14518521,14234045,14597620
4302,14218183,13391296,13520874,14157944,13468103,14521039,13479607,14599742,14405719,14579240,13550441,14172289,13323881,13510529
4303,6027793,8856865,6041984,13831462,11669134,7104526,12898589,13765850,10142625,12195356,12719590,11687324,6245833,12739474


In [36]:
X_test.drop('league_id', axis=1, inplace=True)

In [69]:
Y_pred = multi_xgb_regression.predict(X_test)
Y_pred = np.where(Y_pred<1, 1, Y_pred)
Y_pred = np.where(Y_pred>14, 14, Y_pred)

In [70]:
submission = pd.DataFrame()

In [71]:
submission['club_id'] = np.matrix.flatten(club_ids.values)

In [72]:
submission['league_rank'] = np.matrix.flatten(Y_pred)

In [73]:
submission

,club_id,league_rank
0,14159129,7.110042
1,14016945,10.225660
2,7733794,1.667902
3,11627008,4.447101
4,12733754,6.701785
...,...,...
60265,13460933,5.723778
60266,10699413,4.835227
60267,14583634,10.190293
60268,10055196,2.515396


In [74]:
submission.to_csv('league_rank_predictions.csv', index=False)